In [1]:
import pandas as pd
import numpy as np


# Shipping and Freight Rate

The most widely applied method of calculating how much it costs to ship a container load from A to B is using  a container index. The common container indices are calculated as $/40ft container. The current datasource used for this analysis is the [Freightos Baltic Index](https://terminal.freightos.com/freightos-baltic-index-global-container-pricing-index/#:~:text=Freightos%20Baltic%20Index%20(FBX)%3A%20Global%20Container%20PRICING%20Index,to%20North%20America's%20West%20Coast.). 

The FBX01 freight container index measures 40′ container prices across key port pairs from China or East Asia to North America’s West Coast. Key ports in the index include Shanghai (PVG) and Ningbo (NGB) in China and Los Angeles (LAX) and Chicago (ORD) in the United States. This index is created in collaboration with the Baltic Exchange.

This trade lane is key for global trade, as China is the 3rd largest global importer, with a total of Y percent of trade to the North American West Coast.

In [41]:
fbx = pd.read_excel('../../data/Freightos Baltic Index.xlsx')
fbx['date'] = fbx['date'].apply(lambda x: pd.to_datetime(x))
fbx['FBX'] = fbx['FBX'].astype(float)

In [42]:
df = pd.read_csv("../../data/portwatch/chokepoints_data_02_26_2024.csv")
bab_el_mandeb = df[
    (df["portname"] == "Bab el-Mandeb Strait") & (df["date"] > "2022-12-26")
]

bab_el_mandeb["date"] = pd.to_datetime(bab_el_mandeb["date"])

C:\Users\sahit\AppData\Local\Temp\ipykernel_18580\3094785068.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bab_el_mandeb["date"] = pd.to_datetime(bab_el_mandeb["date"])


In [43]:
bab_el_mandeb = (
    bab_el_mandeb.groupby(pd.Grouper(key="date", freq="W"))["n_total"]
    .sum()
    .reset_index()
)#.merge(fbx, on = 'date')

In [30]:
from datetime import datetime

conflict_date = datetime(2023, 10, 7)
crisis_date = datetime(2023, 11, 17)

In [75]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend, Span, LinearAxis, Range1d, Label, Spacer, Div
from bokeh.layouts import column

# Call this to display plots inline in a Jupyter Notebook
output_notebook()

data["date"] = pd.to_datetime(data["date"])


# Create a figure
p = figure(
    x_axis_type="datetime",
    plot_width=900,
    plot_height=400,
    title="Comparing Weekly Trends in Freightos Baltic Index and Shipping Volume",
)
p.title.text_font_size = "16pt"

subtitle_text = "Source: Total vessel data from PortWatch and Freight Rate from Freightos Baltic Index"
subtitle = Div(
    text=f"<div style='text-align:center;font-size:12pt;color:gray;'>{subtitle_text}</div>",
    width=1000,
    height=30
)

# Add a subtitle
# subtitle = Label(
#         x=0,
#         y=-50,
#         x_units='screen',
#         y_units='screen',
#         text_font="helvetica",  # Setting font to Helvetica
#         text="Source: Total vessel data from PortWatch and Freight Rate from Freightos Baltic Index",
#         text_font_size="12pt",
#         text_color="gray"
#     )


# Plot multiple lines
p1 = p.line(bab_el_mandeb["date"], bab_el_mandeb["n_total"], line_width=2, color="#B07AA1")
p.y_range = Range1d(start=0, end=600)
p.extra_y_ranges = {"y2": Range1d(start=900, end=4000)}
p.add_layout(LinearAxis(y_range_name="y2", axis_label="y2"), "right")
# p2 = p.line(data["date"], data["countryEvents"], line_width=2, color="#FF9DA7")
p3 = p.line(
    fbx["date"],
    fbx["FBX"],
    line_width=2,
    color="#9C755F",
    y_range_name="y2",
)



legend_items = [
    ("Number of Vessels", [p1]),
    # ("Number of events in Yemen", [p2]),
    ("Freightos Baltic Index ($/40ft)", [p3]),
]

# Create the legend manually
legend = Legend(items=legend_items, location="center")

# Add the legend to the plot, but ensure it's outside the main figure
p.add_layout(legend, "right")

# Customize the plot
p.yaxis.axis_label = "Number of Vessels in Bal El-Mandab Strait"
p.right[0].axis_label = "Freightos Baltic Index ($/40ft)"
p.legend.location = "top_right"
p.yaxis.axis_label_text_font_style = "normal"

p.add_layout(
    Span(
        location=crisis_date,
        dimension="height",
        line_color="#7C7C7C",
        line_width=2,
        line_dash=(4, 4),
    )
)
p.add_layout(
    Span(
        location=conflict_date,
        dimension="height",
        line_color="#7C7C7C",
        line_width=2,
        line_dash=(4, 4),
    )
)

crisis_label = Label(
    y=250,  # Adjust as needed
    x=crisis_date,
    y_units="screen",
    text="Red Sea Crisis",
    render_mode="css",
    background_fill_color="white",
    background_fill_alpha=0.7,
    text_font_size="9pt",
)

conflict_label = Label(
    y=230,  # Adjust as needed
    x=conflict_date,
    y_units="screen",
    text="Middle East Conflict",
    render_mode="css",
    background_fill_color="white",
    background_fill_alpha=0.7,
    text_font_size="9pt",
)

p.add_layout(crisis_label)
p.add_layout(conflict_label)


layout = column(p, subtitle,)
# Show the plot
show(layout)

Loading BokehJS ...